<strong style="color: violet; font-size:50px">Bundesliga Scraper</strong>

<img style="float: left;" src="https://upload.wikimedia.org/wikipedia/en/d/df/Bundesliga_logo_%282017%29.svg">

In [1]:
from selenium import webdriver
import time
from tqdm import tqdm
import pandas as pd

In [2]:
driver = webdriver.Opera(executable_path=r"C:\Users\sharm\Downloads\operadriver_win64\operadriver.exe") ##path to your operadriver executable
secs = 12
web_elements_dict = {"distance": 16, "sprints":17, "intensive-runs":18} ##these are the indices of the divs from the site
dfs = []

In [3]:
for key, value in web_elements_dict.items():
    url = f"https://www.bundesliga.com/en/bundesliga/stats/players/{key}"

    load_xpath = f"/html/body/app-root/wrapper-page/div/div/div/div/div/stats-page/div/div[3]/div/stats-rankings-players/div/div/div[{value}]/button"
    table_div_xpath = f"/html/body/app-root/wrapper-page/div/div/div/div/div/stats-page/div/div[3]/div/stats-rankings-players/div/div/div[{value}]"

    driver.get(url)
    time.sleep(secs)

    load_button = driver.find_element_by_xpath(load_xpath)
    for _ in tqdm(range(55), desc=f"Loading {key} data..."):
        time.sleep(1.7)
        load_button.click()

    print(f"Done. Scraping {key} data...")    
    data = driver.find_element_by_xpath(table_div_xpath).text
    t = data.split("\n")
    player_data_dict = {t[2]: float(t[3])}
    for name, value in zip(t[4:-1:2], t[5:-1:2]):
        name = ''.join([i for i in name if not i.isdigit()])
        player_data_dict[name] = float(value)
    df = pd.DataFrame(list(player_data_dict.items()), columns=["player_name", key])
    dfs.append(df)
    print("Done.")

Loading distance data...: 100%|███████████| 55/55 [01:40<00:00,  1.83s/it]


Done. Scraping data...
Done.


Loading sprints data...: 100%|████████████| 55/55 [01:39<00:00,  1.82s/it]


Done. Scraping data...
Done.


Loading intensive-runs data...: 100%|█████| 55/55 [01:40<00:00,  1.83s/it]


Done. Scraping data...
Done.


In [4]:
df1, df2, df3 = dfs
df = df1.merge(df2, on='player_name').merge(df3, on='player_name')
df.head()

,player_name,distance,sprints,intensive-runs
0,Ellyes Skhiri,396.2,586.0,2536.0
1,Maximilian Eggestein,388.1,550.0,2656.0
2,Theodor Gebre Selassie,374.5,746.0,2516.0
3,Robin Knoche,370.6,385.0,1736.0
4,Marvin Friedrich,370.1,449.0,1959.0


In [5]:
df.to_csv("bundesliga_distance-stats.csv", index=False)